In [ ]:
import pickle
import random
import matplotlib.pyplot as plt
import numpy as np
import cv2
import skimage.measure as measure
import skimage.filters as filters
import skimage.morphology as morphology
import skimage.exposure as exposure
from functions import *

In [ ]:
# Name of model
name_model = "Final (M0_6(3x3)_2)"
#version = ".3"

# Path to model
filepath_dic = "Results/" + name_model
#filepath_subdic = filepath_dic + "/" + name_model + version

# Path to prediction
name_val_prediction = filepath_dic + "/val_prediction.pkl"

In [ ]:
# Load predictions (of validation set)
with open(name_val_prediction, 'rb') as file:
    val_predictions = pickle.load(file)

# Load coordinates of annotations (ground truth) (of validation set)
with open("Data/val_coordinates.pkl", 'rb') as file:
    val_coordinates = pickle.load(file)
    
# Load genera (of validation set)
with open("Data/val_genera.pkl", 'rb') as file:
    val_genera = pickle.load(file)

# load validation set (Images and targets)
fileHDFVal = 'Data/val.hdf5'
val = h5py.File(fileHDFVal, 'r')

In [ ]:
print(len(val_predictions))
print(len(val_coordinates))
print(len(val_genera))

# Optimalization parameters

In [ ]:
blurring = True
blur_kernel_size = 11

kernel_shape = "ellipse"

threshold_technique = cv2.THRESH_BINARY+cv2.THRESH_OTSU
#threshold_technique = cv2.THRESH_BINARY
threshold = 1

morphological_operations = {"erosion":{"kernel_size":11, "iterations":1},
                            "dilation":{"kernel_size":11, "iterations":1},
                            "opening":{"kernel_size":15, "iterations":1},
                            "closing":{"kernel_size":11, "iterations":1}}

order_morphological_operations =  ["closing","erosion"] 

## Optimalization based on evaluation metrics

### Post-processing

In [ ]:
rec = []
prec = []
f1 = []
 
for i,prediction in enumerate(val_predictions):
    
    genus = val_genera[i]
    
    # normalize pixel values between 0 and 255
    normalized = normalization(prediction)
    normalized = (normalized*255).astype("uint8")
    
    # post-processing
    post_processed  = post_processing(
        im=normalized,
        blurring=blurring,
        blur_kernel_sz=blur_kernel_size,
        thresh_technique=threshold_technique,
        thresh=threshold,
        kernel_shape=kernel_shape,
        morph_ops=morphological_operations,
        order_morph_ops=order_morphological_operations)
    
    # coordinates of chromosomes
    # predicted
    predicted_positions = centroid(post_processed)
    # real
    real_positions = val_coordinates[i]
    
    #precision
    prec.append(precision_evaluation(coordinates_real=real_positions, 
              coordinates_predicted=predicted_positions,
              genus=genus)) 
    # recall
    rec.append(recall_evaluation(coordinates_real=real_positions, 
              coordinates_predicted=predicted_positions,
              genus=genus))
    
    # f1
    f1.append(F1_evaluation(coordinates_real=real_positions, 
              coordinates_predicted=predicted_positions,
              genus=genus)) 

In [ ]:
print("{:20s}{:^10s}{:^10s}{:^10s}{:^10s}".format("", "MEAN", "MEDIAN","MIN","MAX"))
print("{:20s}{:^10.2f}{:^10.2f}{:^10.2f}{:^10.2f}".format("precision",np.mean(prec)*100,np.median(prec)*100, np.min(prec)*100,np.max(prec)*100))
print("{:20s}{:^10.2f}{:^10.2f}{:^10.2f}{:^10.2f}".format("recall",np.mean(rec)*100, np.median(rec)*100,np.min(rec)*100,np.max(rec)*100))
print("{:20s}{:^10.2f}{:^10.2f}{:^10.2f}{:^10.2f}".format("F1-score",np.mean(f1)*100,np.median(f1)*100,np.min(f1)*100,np.max(f1)*100))

precision = of all predictions how many are correct?

recall = of all postions how many are predicted?

precision high => predicted positions are correct

recall low => a lot of positions are not predicted (due to post-processing or the model did not predict the chromosomes)


In [ ]:
f1_bad = [i for i, x in enumerate(f1) if x < 0.8]     
rec_bad = [i for i, x in enumerate(rec) if x < 0.8]                
prec_bad = [i for i, x in enumerate(prec) if x < 0.8]

In [ ]:
f1_good = [i for i, x in enumerate(f1) if x == 1]       
rec_good = [i for i, x in enumerate(rec) if x == 1]             
prec_good = [i for i, x in enumerate(prec) if x == 1]

In [ ]:
print(f1_bad)
print(rec_bad)
print(prec_bad)

In [ ]:
print(f1_good)
print(rec_good)
print(prec_good)

In [ ]:
bad = f1_bad + rec_bad + prec_bad
bad = list(set(bad))

In [ ]:
good = f1_good + rec_good + prec_good
good = list(set(good))

In [ ]:
bad.sort()
print(bad)

In [ ]:
good.sort
print(good)

In [ ]:
for index in prec_good:
    if rec[index] < 0.8:
        print(index, ":", rec[index])

## Visual optimalization

In [ ]:
#s = random.sample(range(len(val_predictions)),5)
s = rec_good[0:10]

for sample in s:
    plots = []
    
    image = val["features"][sample]
    prediction = val_predictions[sample]
    real_positions = val_coordinates[sample]
    
    # normalization
    normalized = normalization(prediction)
    normalized = (normalized*255).astype("uint8")
    plots.append(normalized)
    
    # blurring
    if blurring:
        blur_kernel = np.ones((blur_kernel_size, blur_kernel_size), np.float32)/blur_kernel_size**2
        blurred = cv2.filter2D(src=normalized, ddepth=-1, kernel=blur_kernel)
        plots.append(blurred)
    else:
        blurred = normalized

    # thresholding
    # thresholding
    if str(threshold_technique) == cv2.THRESH_BINARY+cv2.THRESH_OTSU:
        T = 255
    else:
        T = threshold
    binary = cv2.threshold(blurred,0,T,threshold_technique)[1]
    plots.append(binary)
        
    """n_pixels , intensiteiten = exposure.histogram (blurred)
    plt.figure()
    plt.plot ( intensiteiten , n_pixels )
    plt.xlim (xmin=0.1,xmax=0.2)
    plt.xlabel ("Intensity")
    plt.ylabel ("Number of pixels")
    plt.tight_layout ()"""

    # morpholoical operations
    for morph_op in order_morphological_operations:
        kernel_size = morphological_operations[morph_op]["kernel_size"]
        it = morphological_operations[morph_op]["iterations"]
        if kernel_shape == "ellipse":  
            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(kernel_size, kernel_size))
        elif kernel_shape == "square":
            kernel = np.ones((kernel_size, kernel_size), np.uint8)
        if morph_op=="erosion":
            binary = cv2.erode(binary, kernel, iterations=it)
        if morph_op=="dilation":
            binary = cv2.dilate(binary, kernel, iterations=it)
        if morph_op=="opening":
            binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN,kernel, iterations=it) 
        if morph_op=="closing":
            binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE,kernel, iterations=it) 
        plots.append(binary)
        
    # labbeling
    labelled = morphology.label(binary)
    
    # figure
    plt.figure(figsize=(100,80))
    for i,plot in enumerate(plots):
        plt.subplot(1,len(plots)+2,i+2)
        plt.imshow(plot, interpolation='none')
        plt.scatter(*zip(*real_positions),s=5, c="orange")
        plt.axis("off")
    plt.subplot(1,len(plots)+2,1)
    plt.imshow(image, cmap="gray")
    plt.scatter(*zip(*real_positions),s=5, c="orange")
    plt.axis("off")
    plt.subplot(1,len(plots)+2,len(plots)+2)
    plt.imshow(labelled, cmap="nipy_spectral")
    plt.scatter(*zip(*real_positions),s=5, c="gray")
    plt.scatter(*zip(*centroid(labelled)),s=5, c="w", marker="X")
    plt.axis("off")
    

In [ ]:
i = 72

plots = [val["features"][i], val["targets"][i], val_predictions[i]]

plt.figure(figsize=(100,75))
for j in range(3):
    plt.subplot(1,3,j+1)
    plt.imshow(plots[j], interpolation='none')
    if j == 0:
        plt.scatter(*zip(*val_coordinates[i]),s=50, c="orange")

In [ ]:
plt.imsave(os.path.join("Images thesis", "Target.png"), arr= val["targets"][1], cmap="nipy_spectral", format="png")